In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
llm_name = "gpt-3.5-turbo"


In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

persist_directory = 'data/chroma/'
embedding = OpenAIEmbeddings()
bookdb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
bookdb._collection.count()

180

In [19]:
question = "why lean?"
docs = bookdb.similarity_search(question, k=5)
docs

[Document(page_content='website we list a toll-free number that forwards to our mobile phones and on our business cards\neach of us list our mobile numbers. We emphasize to our customers that they can get in touch', metadata={'page': 8, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}),
 Document(page_content='website we list a toll-free number that forwards to our mobile phones and on our business cards\neach of us list our mobile numbers. We emphasize to our customers that they can get in touch', metadata={'page': 8, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}),
 Document(page_content='disagreement, meetings, and delays. These things can kill morale and lower your chances of\nsuccess.\nHow often have you found yourself stuck on a single design or code element for a whole day?\nHow often have you realized that the progress you made today wasn’t real progress? This\nhappens when you focus on details too early in the process. There’s plenty of time to be a\n

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [21]:
from langchain.chains import RetrievalQA

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=bookdb.as_retriever()
)

In [23]:
result = qa_chain.invoke({"query": question})
result["result"]

'Lean principles are often applied in business and project management to streamline processes, reduce waste, and improve efficiency. By focusing on delivering value to customers, eliminating unnecessary steps, and continuously improving, organizations can become more agile and responsive to changes in the market. Lean methodologies can help teams work more effectively, reduce costs, and deliver higher quality products or services.'

In [24]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [25]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=bookdb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

result = qa_chain.invoke({"query": question})
result["result"]

'Lean methodologies focus on eliminating waste, improving efficiency, and continuously improving processes. By implementing lean practices, businesses can streamline operations, reduce costs, and deliver more value to customers. Thanks for asking!'

In [26]:
result["source_documents"]

[Document(page_content='website we list a toll-free number that forwards to our mobile phones and on our business cards\neach of us list our mobile numbers. We emphasize to our customers that they can get in touch', metadata={'page': 8, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}),
 Document(page_content='website we list a toll-free number that forwards to our mobile phones and on our business cards\neach of us list our mobile numbers. We emphasize to our customers that they can get in touch', metadata={'page': 8, 'source': 'data/getting-real/getting-real-03-stay-lean.pdf'}),
 Document(page_content='disagreement, meetings, and delays. These things can kill morale and lower your chances of\nsuccess.\nHow often have you found yourself stuck on a single design or code element for a whole day?\nHow often have you realized that the progress you made today wasn’t real progress? This\nhappens when you focus on details too early in the process. There’s plenty of time to be a\n

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=bookdb.as_retriever()
)

In [35]:
question = "Is lean a book topic?"
result = qa_chain.invoke({"query": question})
result["result"]

'Yes, "lean" can be a topic discussed in books, particularly in the context of lean methodology, lean manufacturing, or lean principles in business management. It often refers to the concept of maximizing value while minimizing waste.'

In [37]:
# no memory of previous question
question = "why that is needed?"
result = qa_chain.invoke({"query": question})
result['result']

'It seems like you might be referring to the need for a simple and efficient project management tool like Basecamp or a fresh approach to email like HEY. These tools are designed to streamline communication, task management, and collaboration within a team, especially when working with constraints like a small team, different time zones, and limited resources. By using tools like Basecamp or HEY, teams can stay organized, focused, and productive despite challenges like those mentioned in the context.'

In [27]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [29]:
from langchain.chains import ConversationalRetrievalChain
retriever=bookdb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [31]:
question = "Is lean a book topic?"
result = qa.invoke({"question": question})
result['answer']

'Lean is commonly discussed in books related to business management, entrepreneurship, and product development. The concept of lean principles and lean thinking is often applied in these contexts to streamline processes, eliminate waste, and improve efficiency. Lean methodologies are frequently used in industries such as manufacturing, software development, and healthcare to optimize operations and deliver value to customers.'

In [32]:
question = "why that is needed?"
result = qa.invoke({"question": question})
result['answer']

'Lean principles are commonly discussed in books related to business management, entrepreneurship, and product development because they emphasize efficiency, minimizing waste, and continuous improvement. By focusing on delivering value to customers while reducing unnecessary processes and costs, lean methodologies help businesses operate more effectively and adapt to changing market conditions. This approach is particularly valuable in entrepreneurial ventures and product development where resources are often limited, and the ability to pivot quickly based on real-time information is crucial for success.'